# timectrl demo

In [1]:
import pandas as pd
from typing import Literal
import pydantic
from pydantic import BaseModel as PydanticBaseModel
from datetime import date

class BaseModel(PydanticBaseModel):
    class Config:
        arbitrary_types_allowed = True

class SpreadDf(BaseModel):
    amounts: pd.Series
    new_dates: pd.Series

class OrigDf(BaseModel):
    amounts: pd.Series
    dates: pd.Series
    
class CollapsedDf(BaseModel):
    amounts: pd.Series
    closest_new_date: pd.Series


class Spreader:
    def __init__(self):
        pass

    @staticmethod
    def _closest_old_date(new_dates, old_dates):        
        # add trailing ending date to old_dates 
        trailing_date = max(old_dates) + pd.Timedelta(365, unit='D') # TODO: use max spread period
        old_dates = pd.Series(list(old_dates) + [trailing_date])
        # TODO: add spread boundaries on 1/1/yyyy
        return pd.cut(pd.Series(new_dates), pd.Series(old_dates), right=False).apply(lambda x: x.left) # change per arrears/adv # e.g. right=False/True makes a diff here 

    @staticmethod
    def _spread_sum_evenly(amounts, n):
        return pd.Series(sum(amounts)/n for i in range(n))
    
    @staticmethod
    def _repeat_average(amounts, n):
        return pd.Series([sum(amounts)/len(amounts)] * n)
    
    @staticmethod
    def _repeat_last(amounts, n):
        return pd.Series([amounts[-1]] * n)
    
    def _spread_amounts(self, amounts, n):
        return self._spread_sum_evenly(amounts, n)
    
    def _validate_spread_inputs(self, collapsed_df, new_dates):
        """
        Check that all dates in collapsed_df are in new_dates
        """
        assert collapsed_df.closest_new_date.isin(new_dates).all(), "All dates in collapsed_df must be in new_dates"

    
    def get_spread_df(self, collapsed_df: CollapsedDf, new_dates: pd.Series, max_spread_boundaries: Literal["Y", "M"]="Y") -> SpreadDf:
        self._validate_spread_inputs(collapsed_df, new_dates)
        
        old_df = pd.DataFrame({
            'dates':collapsed_df.closest_new_date, 
            'amounts': collapsed_df.amounts})
        # TODO: 
        if max_spread_boundaries == "Y":
            # TODO: implement
            pass
        elif max_spread_boundaries == "M":
            # TODO: implement
            pass

        new_df = pd.DataFrame({
            'new_dates': new_dates ,
            'closest_old_date': self._closest_old_date(new_dates, collapsed_df.closest_new_date),
            'amounts': 0})
        for group, group_df in new_df.groupby('closest_old_date', observed=True).groups.items():
            spread_amounts = self._spread_amounts(
                old_df[old_df['dates'] == group]['amounts'], len(group_df))
            new_df.loc[group_df, 'amounts'] = list(spread_amounts)
        return SpreadDf(amounts = new_df['amounts'], new_dates = new_df['new_dates'])



class Collapser:
    def __init__(self):
        pass

    @staticmethod
    def _use_sum(amounts):
        return amounts.sum()
    
    @staticmethod
    def _use_first(amounts):
        return amounts[0]
    
    @staticmethod
    def _use_last( amounts):
        return amounts[-1]
    
    @staticmethod
    def _use_max(self, amounts):
        return amounts.max()
    
    @staticmethod
    def _use_mean(amounts):
        return amounts.mean()
    
    @staticmethod
    def _use_min(amounts):
        return amounts.min()
    
    def _closest_new_date(self, old_dates, new_dates):
        # assume payments in arrears so old dates after new date could still apply to last new_date, so add trailing new_date
        # TODO: add trailing new_date
        trailing_date = max(new_dates) + pd.Timedelta(365, unit='D') # TODO: use max spread period
        new_dates = pd.Series(list(new_dates) + [trailing_date])
        # collate new_dates wit 1/1/ of each year between new_dats min and trailing date 
        date_boundaries = pd.date_range(start=date(min(new_dates).year, 1,1), end=date(trailing_date.year, 1, 1), freq='Y')
        new_dates = pd.Series(sorted(set(list(new_dates) + list(date_boundaries))))
        return pd.cut(old_dates, new_dates, right=False).apply(lambda x: x.left) # TODO: change per arrears/adv

    def _collapse_amounts(self, amounts):
        return self._use_sum(amounts)
    
    def get_collapsed_df(self, orig_df: OrigDf, new_dates: pd.Series) -> CollapsedDf:
        old_df = pd.DataFrame(
            {'dates': orig_df.dates, 
             'amounts': orig_df.amounts,
            })
            
        old_df['closest_new_date'] = pd.to_datetime(self._closest_new_date(old_df['dates'], new_dates))
        collapsed_amounts = old_df.groupby('closest_new_date').apply(
            lambda x: pd.Series({'amounts': self._collapse_amounts(x['amounts'])}), include_groups=False).reset_index()
        
            
        return CollapsedDf(
            amounts = collapsed_amounts['amounts'], 
            closest_new_date = collapsed_amounts['closest_new_date'])

    
class Timectrl:
        
    def __init__(self, spreader, collapser):
        self.spreader = spreader
        self.collapser = collapser
                
    def get_new_df(self, amounts, dates, new_dates, max_spread_boundaries):
        collapsed_df = self.collapser.get_collapsed_df(
            OrigDf(amounts=pd.Series(amounts), dates=pd.Series(dates)), 
            pd.to_datetime(new_dates))
        new_df = self.spreader.get_spread_df(collapsed_df, pd.to_datetime(new_dates), max_spread_boundaries)
        return new_df 

### implementations

# advance rate spreader/collapser for year boundaries
class rate_spread(Spreader):
    def _spread_amounts(self, amounts, n):
        return self._repeat_average(amounts, n)
    
class rate_collapse(Collapser):
    def _collapse_amounts(self, amounts):
        return self._use_mean(amounts)
    def _closest_new_date(self, old_dates, new_dates):
        return pd.cut(pd.Series(old_dates), pd.Series(new_dates), right=False).apply(lambda x: x.left)
    
rate_transformer = Timectrl(spreader = rate_spread(), collapser = rate_collapse())
new_df = rate_transformer.get_new_df(
    amounts= [1, 2, 3, 4], 
    dates =['2019-01-01', '2019-01-06', '2019-01-07', '2019-01-08'], 
    new_dates=['2019-01-01',  '2019-01-02', '2019-01-03', '2019-01-04','2019-01-10'], 
    max_spread_boundaries = "Y")

print(new_df)

# arrears payments annual boundaries

# advance payments annual boundaries



amounts=0    1.0
1    1.0
2    1.0
3    3.0
4    3.0
Name: amounts, dtype: float64 new_dates=0   2019-01-01
1   2019-01-02
2   2019-01-03
3   2019-01-04
4   2019-01-10
Name: new_dates, dtype: datetime64[ns]


/tmp/ipykernel_12273/1902047314.py:79: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.0, 1.0, 1.0]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new_df.loc[group_df, 'amounts'] = list(spread_amounts)
